# Computational and Artifical Intelligenge

Problem Solving by Searching using IDA* and Best First Search

Things to follow

1. Use appropriate data structures for the fringes and explain the reasoning behind the usage. You need not create data structures, instead use available libraries directly.

2. Avoid any hard-coding unless absolutely necessary.

3. Provide proper documentation

Coding begins now!!!

Define the environment representation in the following code block

# Environment representation goes here

In [210]:
### Algorithm reference reading: https://en.wikipedia.org/wiki/Iterative_deepening_A*
import sys

# grid represented as a 11x8 binary matrix. 0 = obstacle in the cell, 1 = no obstacle
grid = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1],
        [1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1],
        [1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

# grid dimensions
gridLimit = (len(grid) - 1, len(grid[0]) - 1)

# start and end cells in the grid
start = (0, 3)
end = (5,10)

"""configures the modes that can be taken from the current node"""
allowedMoves = [(0, -1), (0, 1), (-1, 0), (1, 0)] #allowed to move left, right, top and bottom

Define the fringe (data structure) and its supported methods in the following code block

# Fringe representation and its associated methods

**Heuristic function** used: h(x) = **Manhatten Distance** between node and destination
**Note**: As we are **allowed to move only left, right, up and down only** - Manhatten distance is a good heuristic
If, we were to move diagonally that straight line distance would be the best heuristic

Conditions to be fullfilled for a Heuristic to be consistent:
    1. H(goal) = 0
    2. H(node) <= cost(node, another_node) + H(another_node)
    
Using a Manhatten distance for H(x); we find that:
    1. Manhatten distance of goal node to goal node = 0.
    2. Manhatten distance complies with triagle inequality, i.e. condition 2 is satisfied. Example:
        if Goal = (5,10), node = (4,5); for any P(4,6):
            H(node) = |5-4| + |10-5| = 6
            cost(node,P) = |4-4| + |5-6| = 1
            H(P) = |5-4| + |10-6| = 5
            it can be found from the above thatL H(node) <= cost(node,P) + H(p)
Hence its proved that Manhatten distance for the given problem is **consistent**.
A consitent heuristic is inherently also **admissible**.

In [211]:
#Node class to store the position, linkage to parent node and costs

class Node():
    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.f = 0
        self.g = 0
        self.h = 0

    def __eq__(self, node):
        return self.position == node.position

#Heuristic function is the Manhattan distance between the node(n) and the goal
def heuristic(n, goal):
    return abs(n.position[0] - goal.position[0]) + abs(n.position[1] - goal.position[1])

#checks if the node is on the path already traversed
def is_in_path(node, path):
    for n in path:
        if (n.position == node.position):
            return True
    return False

#returns the children of a node
def getChildren(node):
    children = []

    for new_position in allowedMoves:
        # Get node position
        node_position = (node.position[0] + new_position[0], node.position[1] + new_position[1])

        # Make sure within range
        if (node_position[0] > gridLimit[0]) or (node_position[0] < 0) or (node_position[1] > gridLimit[1]) or (
                node_position[1] < 0):
            continue

        # Make sure walkable terrain
        if grid[node_position[0]][node_position[1]] != 1:
            continue

        # Create new node
        new_node = Node(node, node_position)

        # Append
        children.append(new_node)

    return children

Define your IDA* in the following code block

# Algorithm IDA*

In [212]:
### Algorithm reference reading: https://en.wikipedia.org/wiki/Iterative_deepening_A*

#Recursive function that checks for the next best heuristic and returns a dictionary containing the path, costs and bound limit
# nodecount and nodesinmem used to track the nodes expanded and the nodes in memory
def astarSearch(path, g, bound, goal_node, nodesExpanded, nodesinMem):
    node = path[len(path)-1]
    f = g + heuristic(node, goal_node)

    # the node is higher than the bounding limit
    if (f > bound):
        return {'bound': f, 'path': path}
 
    #found the goal node
    if (node.position == goal_node.position):
        return {'bound': -1, 'path': path, 'cost': f, 'nodesExpanded': nodesExpanded, 'nodesinMem': nodesinMem }

    # assign to max value to help find the lowest higher heuristic than the bound value
    min = sys.maxsize

    #expand the node
    nodesExpanded = nodesExpanded + 1
    
    #loop though the children
    for child in getChildren(node):
        if is_in_path(child, path) == False:
            path.append(child)
            
            #recursively call the search function with new heuristic
            result = astarSearch(path, g + heuristic(node, child), bound, goal_node, nodesExpanded, nodesinMem + 1)
            
            if result['bound'] == -1: #goal has been found
                return {'bound':-1, 'path':path, 'cost': result['cost'], 'nodesExpanded': result['nodesExpanded'], 'nodesinMem': result['nodesinMem'] }
            if result['bound'] < min: # found a child with lower bounding value
                min = result['bound']
            path.pop()

    return {'bound': min, 'path':path}

# the main IDA* search function
def ida(grid, start, goal):
    #initialize the start parameters
    bound = heuristic(start, goal)
    path = [start_node]
    g = 0

    #loop until we find the goal or fail
    while 1:
        result = astarSearch(path, 0, bound, goal, 0, 0)

        if (result['bound'] == -1):    #found
            return result
        if (result['bound'] == sys.maxsize): #not found
            return result

        bound = result['bound']

Define your Greedy Best First Search algorithm in the following code block

# Algorithm - Best First Search

In [213]:
# The heuristic function, Node class are reused form the ones defined above. An iterative implementation
def bestFirstSearch(grid, start, goal):

    # Initialize both open and closed list
    open_list = []
    closed_list = []
    
    nodesExpanded = 1
    nodesInMemory = 1
    cost = 0
    
    #grid dimensions
    gridLimit = (len(grid)-1, len(grid[0])-1)

    # Add the start node
    open_list.append(start_node)

    # Loop as long as there are nodes to expand
    while len(open_list) > 0:
        
        # pick the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index

        # Move current from open list to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)
        
        nodesExpanded = nodesExpanded + 1

        # Found the goal node
        if current_node == goal_node:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return {'path': path[::-1], 'cost': cost, 'nodesExpanded':nodesExpanded, 'nodesInMemory': nodesInMemory} # Return the results

        #print ('Generating childred for:', current_node.position, current_node.f, current_node.g, current_node.h)
        # Generate children
        children = []
        for new_position in allowedMoves: # Adjacent squares

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range
            if (node_position[0] > gridLimit[0]) or (node_position[0] < 0) or (node_position[1] > gridLimit[1]) or (node_position[1] < 0):
                continue

            # Make sure walkable terrain
            if grid[node_position[0]][node_position[1]] != 1:
                continue

            # Create new node
            new_node = Node(current_node, node_position)
            nodesInMemory = nodesInMemory + 1
            
            # Append
            children.append(new_node)

        # Loop through children
        for child in children:

            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    continue

            # Create the f, g, and h values
            child.h = heuristic(child, goal_node)
            child.f = child.h

            # Child is already in the open list
            for i, open_node in enumerate(open_list):
                if child == open_node:
                    if child.f > open_node.f:
                        continue
                    else:
                        open_list[i] = child
                        continue

            # Add the child to the open list
            open_list.append(child)
        
        #increment the cost as we take the next step
        cost = cost + 1

Feel free to add code blocks for any other methods needed starting here.

In [214]:
#Code Block 1


Call your main function/algorithm for IDA* in the next code block with appropriate input representation

## Computation call for IDA*

In [215]:
# Initialize the start and goal nodes
start_node = Node(None, start)
goal_node = Node(None, end)

#call the ida function
idaResult = ida(grid, start_node, goal_node)

Call your main function/algorithm for Best First Search in the next code block with appropriate input representation

## Computation call for Best First Search

In [216]:
# use the same start and goal nodes for bestfirstseach
bfsResult = bestFirstSearch(grid, start_node, goal_node)

The agent should provide expected output for questions mentioned below in the subsequent blocks

## (1) Path taken to reach destination from source for IDA*

In [217]:
# Execute statement to retrieve the path taken here using IDA*
if idaResult['bound'] == -1:
    nodes = []
    for node in idaResult['path']:
        nodes.append(node.position)

    print('IDA* Path:', nodes)
else:
    print ('IDA* Path Not Found!')

IDA* Path: [(0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (1, 10), (2, 10), (3, 10), (4, 10), (5, 10)]


## (1) Path taken to reach destination from source for Best First Search

In [218]:
# Execute statement to retrieve the path taken here using Best First Search
print('BFS Path:', bfsResult['path'])

BFS Path: [(0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (1, 10), (2, 10), (3, 10), (4, 10), (5, 10)]


## (2) Cost of the path for IDA* here

In [219]:
# Execute statement to retrieve the cost of the path here using IDA*
if idaResult['bound'] == -1:
    print('IDA* Cost:', idaResult['cost'])
else:
    print ('IDA* Path Not Found!')

IDA* Cost: 12


## (2) Cost of the path for Best First Search here

In [220]:
# Execute statement to retrieve the cost of the path here using Best First Search
print('BFS Cost:', bfsResult['cost'])

BFS Cost: 12


## (3) Total Number of nodes expanded to get this state using IDA*

In [221]:
# Execute statement to retrieve the total number of nodes expanded to get this state using IDA* here
print('IDA* nodes expanded:', idaResult['nodesExpanded'])

IDA* nodes expanded: 12


## (3) Total Number of nodes expanded to get this state using Best First Search

In [222]:
# Execute statement to retrieve the total number of nodes expanded to get this state using Best First Search here
print('BFS nodes expanded:', bfsResult['nodesExpanded'])

BFS nodes expanded: 14


## (4) Maximum number of nodes kept at the memory at any point in time using IDA*

In [223]:
# Execute Output for the maximum number of nodes kept at the memory at any point in time using IDA* here
print('IDA* nodes in memory:', idaResult['nodesinMem'])

IDA* nodes in memory: 12


## (4) Maximum number of nodes kept at the memory at any point in time using Best First Search

In [224]:
## Execute Output for the maximum number of nodes kept at the memory at any point in time using Best First Search here
print('BFS nodes in memory:', bfsResult['nodesInMemory'])

BFS nodes in memory: 33


All the best!! Happy Coding!!
Let human intelligence prevail

# Closure Notes:
Given,  
    b: branching factor  
    d: depth of first solution  

**IDA* Complexity:**  
    Time complexity: O(b^d)  
    Space complexity: O(d)  

**BFS Complexity:**  
    Time complexity: O(b^d)  
    Space complexity: O(bd)  
    
IDA* keeps less nodes in memory compared to A* or BFS